In [1]:
from bokeh.plotting import figure, show, Column
from bokeh.io import output_notebook
from bokeh.models import DataTable, TableColumn, PointDrawTool, ColumnDataSource
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead, LabelSet,Slider,CustomJS

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
from bokeh import events
from bokeh.models import Div
from bokeh.layouts import column, row

In [4]:
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
]

p = figure(
    x_range=(-5, 5), y_range=(-5, 5),
    title='Complex tool',
    #tooltips = TOOLTIPS,
)

div = Div(width=200, height=p.plot_height, height_policy="fixed")

x = [-3, 3, 1.5]
y = [ 1,-2, 4]
s = ColumnDataSource(dict(
    x = x,
    y = y,
    color = ['black', 'black', 'black'],
    names = ['a','z','b']
))

s2 = ColumnDataSource(data=dict(x=x, y=y))
p.line(x='x', y='y', color="#82d93f", line_width=2, alpha=0.6, source=s2)

#s3 = ColumnDataSource(data=dict(x=x, y=y))
#p.line(x='x',y='y',line_width=2,source=s3)

labels = LabelSet(x='x', y='y', text='names', level='glyph',
              x_offset=5, y_offset=5, source=s, render_mode='canvas')

renderer = p.scatter(x='x', y='y', source=s, color='color', size=10)

callback = CustomJS(args=dict(s=s,s2=s2), code="""
    var data = s.data;
    var data2 = s2.data;
    y = data['y']
    x = data['x']
    x2 = data2['x']
    y2 = data2['y']

    for (i = 0; i < x.length; i++) {
        y2[i] = y[i]
        x2[i] = x[i]
    }
    //y2.push(y[0]);
    //x2.push(x[0]);
    
    s.change.emit();
    s2.change.emit();
    """)

draw_tool = PointDrawTool(renderers=[renderer], empty_value='black',add=False)

s.js_on_change('data',callback)

p.add_tools(draw_tool)
p.toolbar.active_tap = draw_tool

arr = Arrow(end=NormalHead(fill_color="orange"),
                   x_start=1, y_start=0, x_end=0.2, y_end=0.7)
#p.add_layout(arr)

div = Div(
    text="""Your <a href="https://en.wikipedia.org/wiki/HTML">HTML</a>
    -supported text is initialized with the <b>text</b> argument.  The
    remaining div arguments are <b>width</b> and <b>height</b>.
    For this example, those values are <i>200</i> and <i>100</i>
    respectively.""",
width=600, height=50)
p.add_layout(labels)
p.toolbar.active_drag = draw_tool
#p.toolbar.tools = None


show(column(div,p))